# ARIMA models in Python

* Modeling wait times in hospitals? Emergency rooms?

Course Description: Have you ever tried to predict the future? What lies ahead is a mystery which is usually only solved by waiting. In this course, you will stop waiting and learn to use the powerful ARIMA class models to forecast the future. You will learn how to use the statsmodels package to analyze time series, to build tailored models, and to forecast under uncertainty. How will the stock market move in the next 24 hours? How will the levels of CO2 change in the next decade? How many earthquakes will there be next year? You will learn to solve all these problems and more.

## ARMA models
* Time series are everywhere:
    * Science
    * Technology
    * Business
    * Finance
    * Policy
* ARIMA models are one of the go-to time-series tools
* **Trend:** a positive trend is a line that generally slopes up; a negative trend is a line that generally slopes down
* **Seasonality:** has patterns that repeat at regular intervals, for example high sales every weekend
* **Cyclicality:** in contrast to seasonality, has a repeating pattern but no fixed periods/time intervals.
* **White noise:** has uncorrelated values

#### Stationarity
* To model a time series, it must be stationary
* **Stationary:** Means that the distribution of the data doesn't change with time. For time series to be stationary, it must fulfill three criteria:
    * **Trend stationary:** series has zero trend
    * **Variance is constant:** the avererage distance of the data points from the zero line isn't changing
    * **Autocorrelation is constant:** how each value in the time series is related to its neighbors stays the same.
* For train-test split, the data must be split in time (not shuffled or reordered)
* We train on the data earlier in the time series and test on the data that comes later


#### Making time series stationary:
#### Augmented Dickey-Fuller Test:
* tests for trend non-stationarity
* Null hypothesis is time series is non-stationary due to trend

```
from statsmodels.tsa.stattools import adfuller
results = adfuller(df['close'])
```
* 0th element: test statistic
    * More negative means more likely to be stationary
* 1st element: p-value
    * If p-value is small (smaller than 0.05) $\Rightarrow$ reject null hypothesis (reject non-stationarity)
* 4th element: dictionary of critical values of the test statistic
    
* **Plotting time series can stop you from making incorrect assumptions and ends up saving you time!

* Remember: the Dickey Fuller test only tests for stationarity.

* Making a time series stationary $\Rightarrow$ A bit like feature engineering in classic ML.

* One very common way to make a time series stationary is to **take first differences.**
* For some time series, **we may need to take the difference more than once.**
* **Sometimes, we will need to perform other transformations to make the time series stationary.**

#### Other transformations
* **Take the log:** `np.log(df)`
* **Take the square root:** `np.sqrt(df)`
* **Take the proportional change:** `df.shift(1)/df`




### Autoregressive (AR) Models
* In an AR(1) model:
    * Today's value = a mean + a fraction ($\phi$) of yesterday's value + noise
    * $R_t$ = $\mu$ + $\phi$$R_{t-1}$ + $\epsilon_t$
* Since there is only 1 lagged value on the right hand side, this is called an AR model of order 1, or simply an AR(1) model.
* If the AR parameter **$\phi$ is 1**, then the process is a **random walk**.
* If **$\phi$ is 0**, then the process is **white noise**.
* In order for the process to be **stable** and **stationary**, $\phi$ has to be between -1 and 1
    * -1 < $\phi$ < 1
* **Negative $\phi$:** Mean reversion
* **Positive $\phi$:** Momentum
* The autocorrelation **decays exponentially at a rate of $\phi$.**
    * This means that if $\phi$ is 0.9:
        * the lag-1 autocorrelation is 0.9 
        * the lag-2 autocorrelation is $0.9^2$
        * the lag-3 autocorrelation is $0.9^3$
        * ... etc. ...
    * When $\phi$ is negative, the autocorrelation function still decays exponentially, but the signs of the autocorrelation function reverse at each lag.
    
* Higher Order AR Models:
    * AR(1) 
        * $R_t$ = $\mu$ + $\phi_1$$R_{t-1}$ + $\epsilon_t$
    * AR(2)
        * $R_t$ = $\mu$ + $\phi_1$$R_{t-1}$ + $\phi_2$$R_{t-2}$ + $\epsilon_t$
    * AR(3)
        * $R_t$ = $\mu$ + $\phi_1$$R_{t-1}$ + $\phi_2$$R_{t-2}$ + $\phi_3$$R_{t-3}$ + $\epsilon_t$
    * etc. ....

#### Simulating an AR Process

```
from statsmodels.tsa.arima_process import ArmaProcess
ar = np.array([1, -0.9])
ma = np.array([1])
AR_object = ArmaProcess(ar, ma)
simulated_data = AR_object.generate_sample(nsample=1000)
plt.plot(simulated_data)
```
* The convention for defining the order and parameters of the AR process is a little counterintuitive:
    * You must include the zero-lag coefficient of 1, and the sign of the other coefficient is the opposite of what we have been using. 
    * For example, for an AR(1) process with $\phi$ = **+0.9**, the second element of the ar array should be the opposite sign, **-0.9**
    
#### Estimating and Forecasting as AR Model
* Statsmodels has another model for estimating the parameters of a given AR model

#### Estimating an AR Model
* To estimate parameters from data (simulated):

```
from statsmodels.tsa.arima_model import ARMA
mod = ARMA(simulated_data, order=(1,0))
result = mod.fit()
```
* The arguments of `mod` are: 1) the data you are trying to fit and 2) the order of the model
    * An order (1,0) would mean you're fitting the data to an AR(2) model.
    * An order (2,0) would mean you're fitting the data to an AR(2) model.
    * The second part of the order is the MA part (discusssed in next chapter).
    * To see the full output, use the summary method on result:
        * `print(result.summary())
            * `const` = $\mu$
            * `ar.L1.y` = $\phi$
    * If you just want to see the coefficients rather than the entire regression output, you can use:
        * `print(result.params)`
        * returns array of the fitted coefficients $\mu$ and $\phi$
        
#### Forecasting an AR Model 
* To do forecasting, both in sample and out of sample, you still create an instance of the class using ARMA, and use `.plot_predict` to do forecasting

```
from statsmodels.tsa.arima_model import ARMA
mod = ARMA(simulated_data, order=(1,0))
res = mod.fit()
res.plot_predict(start='2016-07-01', end='2017-06-01')
plt.show()
```

#### Choosing the Right Model
* In practice, you will ordinarily not be told the order of the model that you're trying to estimate
* Two techniques to determine order: 
    * The **Partial Autocorrelation Function (PACF):** measures the incremental benefit of adding another lag
        * **`.plot_pacf`:** same usage as `plt.acf`; is the statsmodels function for plotting the partial autocorrelation function
    * The **Information criteria:** adjusts the goodness-of-fit of a model by imposing a penalty based on the number of parameters used.
    * Two popular adjusted goodness-of-fit measures:
        * **AIC (Akaike Information Criterion)**
        * **BIC (Bayesian Information Criterion):** In practice, the best way to use the information criteria is to fit several models, each with a different number of parameters, and choose the one with the lowest information criterion
        * Both AIC and BIC are included in the full estimation output of an ARMA model (`result.summary()`)
        * To get solely the AIC or BIC statistics:
            * `result.aic`
            * `result.bic`
    
#### PACF
```
from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(x, lags=20, alpha=0.05)
```
* The input `x` is a series or array 
* The argument `lags` indicates how many lags of the partial autocorrelation function will be plotted 
* The `alpha` argument sets the width of the confidence interval



### Moving Average (MA) and ARMA Models
#### Describe Model
* In a moving average, or MA model 
* Mathematical Description of a MA(1) Model:
    * Today's value equals a mean plus noise, plus a fraction of theta of yesterday's noise
    * $R_t = \mu + \epsilon_t + \theta\epsilon_{t-1}$
    * Since there is only one lagged error on the right hand side, this is called an MA model of order 1, or simply an MA(1) model. 
    * If $\theta$ is 0, then the process is white noise
    * MA models are stationary for all models of $\theta$
    * **Negative $\theta$: One-Period Mean Reversion**; a shock two periods ago would have **no** effect on today's return- only the stock now and last period
    * **Positive $\theta$: One-Period Momentum**
    * **Note:** One-period autocorrelation is $\theta / (1 + \theta^2)$, not $\theta$
    
#### Simulating an MA Process

```
from statsmodels.tsa.arima_process import ArmaProcess
ar = np.array([1])
ma = np.array([1, 0.5])
AR_object = ArmaProcess(ar, ma)
simulated_data = AR_object.generate_sample(nsample=1000)
plt.plot(simulated_data)
```
* For an MA(1), the AR order is just an array containing 1 
* The MA order is an array containing 1 and the MA(1) parameter $\theta$
* Unlike with the AR simulation, no need to reverse the sign of $\theta$

```
# import the module for simulating data
from statsmodels.tsa.arima_process import ArmaProcess

# Plot 1: MA parameter = -0.9
plt.subplot(2,1,1)
ar1 = np.array([1])
ma1 = np.array([1, -0.9])
MA_object1 = ArmaProcess(ar1, ma1)
simulated_data_1 = MA_object1.generate_sample(nsample=1000)
plt.plot(simulated_data_1)

# Plot 2: MA parameter = +0.9
plt.subplot(2,1,2)
ar2 = np.array([1])
ma2 = np.array([1, 0.9])
MA_object2 = ArmaProcess(ar2, ma2)
simulated_data_2 = MA_object2.generate_sample(nsample=1000)
plt.plot(simulated_data_2)

plt.show()
```

#### Estimation and Forecasting an MA Model
* Same as estimating an AR model (except for `order=(0,1)`)

```
from statsmodels.tsa.arima_model import ARMA
mod = ARMA(simulated_data, order=(0,1))
result = mod.fit()
```
**Note** that now the order is (0,1) for an MA(1) model, and not (1,0) as for an AR(1) model

#### Forecasting an MA Model
* The procedure for forecasting an MA model is the same as that for an AR model, but set the order to (0,1):

```
from statsmodels.tsa.arima_model import ARMA
mod = ARMA(simulated_data, order=(0,1))
res = mod.fit()
res.plot_predict(start='2016-07-01', end='2017-06-01')
plt.show()
```
* **One thing to note** is that with an MA(1) model, unlike an AR model, all forecasts beyond the one-step ahead forecast will be the same

### ARMA models
* An **ARMA model** is a combination of an AR and MA model.
* Mathematcial equation for an ARMA(1,1) model:
    * $R_t = \mu + \phi R_{t-1} + \epsilon_t + \theta \epsilon_{t-1}$
* an ARMA(1,1) model contains AR(1) and MA(1) model components
* ARMA models can be converted to pure AR or pure MA models

### Cointegration Models
* What is **Cointegration?**
    * Two series, $P_t$ and $Q_t$ can be random walks
    * But the linear combination $P_t - cQ_t$ may not be a random walk!
    * If that's true:
        * $P_t - cQ_t$ is forecastable
        * $P_t$ and $Q_t$ are said to be **cointegrated**
        
* Analogy: **Dog on a (retractable) Leash**
    * $P_t$ = Owner
    * $Q_t$ = Dog
    * Both series (individually) look like a random walk
    * However the difference or distance between them, will be mean reverting.
    * The dog and its owner are linked together and their distance is a mean reverting process
* What types of series are cointegrated?
    * Commodities
    * Competitors are not necessarily economic substitutes/ cointegrated
    
#### Two Steps to Test for Cointegration
* First: Regress the level of one series on the level of the other series, to the get the slope coefficient c
    * Regress $P_t$ on $Q_t$ and get slope $c$
* Secong: Run Augmented Dickey-Fuller test on $P_t - cQ_t$ to test for random walk

* Alternatively:

```
from statsmodels.tsa.stattools import coint
coint(P,Q)
```

## ARIMA vs SARIMAX models in Python
[Medium article](https://medium.com/swlh/a-brief-introduction-to-arima-and-sarima-modeling-in-python-87a58d375def)
* Time series analysis is a great tool for predicting future events such as market values changing. 
* ARIMA and SARIMA require data in a ‘long’ format.
* **ARIMA** stands for **A**uto **R**egressive **I**ntegrated **M**oving **A**verage
* **SARIMAX** is similar and stands for **S**easonal **A**uto **R**egressive **I**ntegrated **M**oving **A**verage with e**X**ogenous factors

* In an auto regressive (AR) model the model predicts the next data point by looking at previous data points and using a mathematical formula similar to linear regression.
* There is something called an **order** (represented by **“p”**) that determines how many previous data points will be used. 
    * The best way to get a good p value is to just try a few different values and see which model came out with the minimum (best) AIC.
    * Next up is to explain the **integrated** part of ARIMA and SARIMAX. For the auto regressive and moving average models to work the data must be stationary. This means the data needs to not have trends or seasonality.
    * **Integration** is taking a difference of the time-series, subtracting the previous value from each value, which tends to make the data more stationary.
    * There is a value called **“d”** which represents **how many times the data is to be differenced.**
        * Like the p value in the auto regressive model, it is best to simply try out a few different values for d and see which model has the minimal AIC.
    * A moving average (MA) model performs calculations based on noise in the data along with the data’s slope. 
    * **Combining AR and MA along with differencing (I) creates the ARIMA model.** 
        * There is a value called **“q”** in the moving average model that is the **order**. The order is similar to the auto regressive order where it is the amount of past data points to put into the equation.
        
        
#### SARIMAX
* SARIMAX is used on data sets that have seasonal cycles. The difference between ARIMA and SARIMAX is the seasonality and exogenous factors (seasonality and regular ARIMA don’t mix well)
    * the key take away is that SARIMAX requires not only the p, d, and q arguments that ARIMA requires, but it also requires another set of p, d, and q arguments for the seasonality aspect as well as an argument called **“s”** which is **the periodicity of the data’s seasonal cycle.**
        * When choosing an s value try to get an idea of when the seasonal data cycles. If your data points are separated by a monthly basis and the seasonal cycle is a year, then set s to 12. Or if the data points are separated by a daily basis and the seasonal cycle is a week then make s equal to 7.
* For all the values other than SARIMAX’s “s” value I recommend using a grid search this can be used with the itertools library in python
    * find the minimum value in the list of AICs and then find that key in the dictionary. The resulting key will be the optimal order to create a model with.
    
* To run an ARIMA model there are two arguments: the actual data and the order. The order is a tuple containing p, d, and q.

```
from statsmodels.tsa.arima_model import ARIMA
import itertools
# Grid Search
p = d = q = range(0,3) # p, d, and q can be either 0, 1, or 2
pdq = list(itertools.product(p,d,q)) # gets all possible combinations of p, d, and q
combs = {} # stores aic and order pairs
aics = [] # stores aics
# Grid Search continued
for combination in pdq:
    try:
        model = ARIMA(data, order=combination) # create all possible models
        model = model.fit()
        combs.update({model.aic : combination}) # store combinations
        aics.append(model.aic)
    except:
        continue
        
best_aic = min(aics)
# Model Creation and Forecasting
model = ARIMA(data, order=combs[best_aic])
model = model.fit()
model.forecast(7)[0]
```

* SARIMAX is much like ARIMA, but a little more complicated. 
* Not only do you have to use a loop and grid search for the optimal values of p, d, and q, but you have to also use a nested loop and grid search for the seasonal values for p, d, and q. 
* There are also many more parameters in the SARIMAX function.

```
import statsmodels.api as sm
import itertools
# Grid Search
p = d = q = range(0,3) # p, d, and q can be either 0, 1, or 2
pdq = list(itertools.product(p,d,q)) # gets all possible combinations of p, d, and q 
p2 = d2 = q2 = range(0, 2) # second set of p's, d's, and q's
pdq2 = list(itertools.product(p2,d2,q2)) # simular too code above but for seasonal parameters
s = 12 # here I use twelve but the number here is representative of the periodicty of the seasonal cycle
pdqs2 = [(c[0], c[1], c[2], s) for c in pdq2]
combs = {}
aics = []
# Grid Search Continued
for combination in pdq:
    for seasonal_combination in pdqs2:
        try:
            model = sm.tsa.statespace.SARIMAX(data, order=combination, seasonal_order=seasonal_combination,
                                             enforce_stationarity=False,
                                             enforce_invertibility=False)
            model = model.fit()
            combs.update({model.aic : [combination, seasonal_combination]})
            aics.append(model.aic)
            
        except:
            continue
            
best_aic = min(aics)
# Modeling and forcasting
model = sm.tsa.statespace.SARIMAX(data, order=combs[best_aic][0], seasonal_order=combs[best_aic][1],
                                             enforce_stationarity=False,
                                             enforce_invertibility=False)
model = model.fit()
model.forecast(7)
```

# DataCamp ARIMA Models with Python

* Time series data:
    * Science
    * Technology
    * Business
    * Finance
    * Policy
    * Medicine/ health/ disease/ public health
    * Energy sector
    * Population growth
    * etc. ...
* Forecast all of these datasets using time series models, and ARIMA models are one of the go-to time series tools. 

#### Trend:
* positive trend slopes up
* negative trend slopes down

```
fig, ax = plt.subplots()
df.plot(ax=ax)
plt.show()
```

#### Seasonality 
* A seasonal time series has patterns that repeat at regular intervals, for example high sales every weekend, or lowest temperatures in the winter. 

#### Cyclicality
* Cyclicality is where there is a repeating pattern but no fixed period

#### White noise
* White noise series has uncorrelated values
* A series of measurement, where each value is uncorrelated with previous values
* Just like flipping a coin, with white noise, the series value doesn't depend on the values that came before

* **To model a time series, it must be stationary** (meaning that the distribution of the data doesn't change with time). 
* **For a time series to be stationary, it must fulfill three criteria:**
    * 1) **The series has zero trend:** it is neither growing nor shrinking
    * 2) **The variance is constant:** The average distance of the data points from the zero line isn't changing\
    * 3) **The autocorrelation is constant:** How each value in the time series is related to its neighbors stays the same.
    
* Generally, in machine learning, you have a training set, which you fit your model on, and a test set, which you will test your predictions against. Time series forecasting is just the same, but the train-test split will be different. 
    * We use the past values to make future predictions, and so we will need to split the data in time. 
    * We train on the data earlier in the time series and test on the data that comes later. 
    * Split time series at a given date as shown below:
        * `df_train = df.loc[:'2018']`
        * `df_test = df.loc['2019':]`

### Making time series stationary
* The most common test for identifying whether a time series is non-stationary or not is the **Augmented Dickey-Fuller Test** aka the **Adfuller Test**

#### The Augmented Dickey-Fuller Test
* Tests for trend non-stationarity
* Null hypothesis is time series is non-stationary

```
from statsmodels.tsa.stattools import adfuller
results = adfuller(df['close'])
```
* The results object is a tuple containing:
    * **Oth element:** the test statistic
        * The more negative this number is, the more likely that the data is stationary
    * **1st element:** test p-value
        * If p-value is small (for example <0.05) $\Rightarrow$ reject null hypothesis. Reject non-stationary.
    * **2nd element:** a dictionary storing the critical values of the test statistic, which equate to different p-values
    
* **It is always worth plotting your time series as well as doing the statistical tests.**
* Plotting time series can stop you making wrong assumptions.
* Remember that the Dickey-Fuller test **only tests for trend stationarity.**

* 1) **Taking the difference:**
    * `df_stationary = df.diff()`
    * By default, will lead to `NaN`s:
    * `df_stationary = df.diff().dropna()`
    * For some time series, you may need to take the difference more than once.
* 2) **Other transforms:**
    * Sometimes, we will need to perform other transformations to make the time series stationary
    * Examples of other transforms/ transformations:
        * Take the **log**:
        * `np.log(df)`
        * Take the **square root**:
        * `np.sqrt(df)`
        * Take the **proportional change**:
        * `df.shift(1)/df`
    * Often, the simplest solution is the best one.

## Intro to AR, MA, and ARMA models
* AR and MA models and how these are combined into ARMA models.

#### AR models
* **Autoregressive** (AR) model
* In an autoregressive model, we regress the values of the time series against previous values of this same time series:
    * First order AR model, or AR(1) model:
    * $y_t = a_1y_{t-1}+\epsilon_t$
* The "shock term", $\epsilon_t$, is white noise.
* $a_1$ is the autoregressive coefficient at lag one
* Compare this to a simple linear regression where the dependent variable is $y_t$ and the independent variable is $y_{t-1}$
* The coefficient $a_1$ is just the slope of the line and the shocks are the residuals to the line
* The **order** of the model is the **number of time lags** used.
* An order two AR model, AR(2) has two autoregressive coefficients, and has two independent variables, the series at lag one, and the series at lag two:
    * $y_t = a_1y_{t-1} + a_2y_{t-2}+\epsilon_t$
* More generally, we use **p** to mean the order of the AR model: **AR(p)**
    * This means we have **p** autoregressive coefficients and us p lags
    
#### MA models
* Moving average (MA) models
* In a **moving average model**, we regress the values of the time series against the **previous shock values** of this same time series
* First order MA model: 
    * $y_t = m_1\epsilon_{t-1}+\epsilon_t$
* The value of the time series is $m_1$ times the value of the shock at the previous step; plus the shock term for the current time step
* The order of the model means how many time lags we use
* An MA(2) model would include shocks from one and two steps ago:
    * $y_t = m_1\epsilon_{t-1}+m_2\epsilon_{t-2}+\epsilon_t$
* More generally, we use **q** to mean the **order of the MA model**.

#### ARMA models
* Autoregressive moving-average (ARMA) model
* ARMA = AR + MA
* An ARMA model is a combination of the AR and MA models
* The time series is regressed on the previous values and the precious shock terms
* An ARMA(1,1) model:
    * $y_t=a_1y_{t-1}+m_1\epsilon_{t-1}+\epsilon_t$
* More generally, we use ARMA-p-q to define an ARMA model

#### ARMA(p, q)
* p is order of AR (autoregressive) part
* q is order of MA (moving-average) part

* Using the statsmodels package, we can both fit ARMA models and create ARMA data.

* Let's take this ARMA-one-one model:
    * $y_t = a_1y_{t-1} + m_1\epsilon_{t-1}+\epsilon_t$
* Say we want to simulate the data with these coefficients:
    * $y_t = 0.5y_{t-1}+0.2\epsilon_{t-1}+\epsilon_t$

* First, we import the arma_generate_sample function
* Then we make lists for the AR and MA coefficients
    * Note that both coefficient lists start with 1.
    * This is for the zero lag term and we will always set this to one (unless we are simulating an ARMA sample of an order other than (1,1)
    * **Note that the AR coefficient sign is reversed** (as always)
* Generate the data, passing in the coefficients, the **number of data points to create**, and the **standard deviation of the shocks**.

```
from statsmodels.tsa.arima_process import arma_generate_sample
ar_coefs = [1, -0.5]
ma_coefs = [1, 0.2]
y = arma_generate_sample(ar_coefs, ma_coefs, nsample=100, scale=0.5)
```

#### Fitting and ARMA model

```
from statsmodels.tsa.arima_model import ARMA
# Instantiate model object
model = ARMA(y, order=(1,1))
# Fit model
results = model.fit()
```

* Remember for any model ARMA(p,q):
    * The list ar_coefs has the form [1, -a_1, -a_2, ..., -a_p].
    * The list ma_coefs has the form [1, m_1, m_2, ..., m_q],
* where a_i are the lag-i AR coefficients and m_j are the lag-j MA coefficients.

#### For MA(1) model with MA lag-1 coefficient of -0.7:
```
# Import data generation function and set random seed
from statsmodels.tsa.arima_process import arma_generate_sample
np.random.seed(1)

# Set coefficients
ar_coefs = [1]
ma_coefs = [1, -0.7]

# Generate data
y = arma_generate_sample(ar_coefs, ma_coefs, nsample=100, scale=0.5)

plt.plot(y)
plt.ylabel(r'$y_t$')
plt.xlabel(r'$t$')
plt.show()
```

#### For AR(2) model with AR lag-1 and lag-2 coefficients of 0.3 and 0.2 respectively:

```
# Import data generation function and set random seed
from statsmodels.tsa.arima_process import arma_generate_sample
np.random.seed(2)

# Set coefficients
ar_coefs = [1, -0.3, -0.2]
ma_coefs = [1]

# Generate data
y = arma_generate_sample(ar_coefs, ma_coefs, nsample=100, scale=0.5)

plt.plot(y)
plt.ylabel(r'$y_t$')
plt.xlabel(r'$t$')
plt.show()
```
**For model with form: $y_t = -0.2y_{t-1}+0.3\epsilon_{t-1}+0.4\epsilon_{t-2}+\epsilon_t$
```
# Import data generation function and set random seed
from statsmodels.tsa.arima_process import arma_generate_sample
np.random.seed(3)

# Set coefficients
ar_coefs = [1, 0.2]
ma_coefs = [1, 0.3, 0.4]

# Generate data
y = arma_generate_sample(ar_coefs, ma_coefs, nsample=100, scale=0.5)

plt.plot(y)
plt.ylabel(r'$y_t$')
plt.xlabel(r'$t$')
plt.show()
```

#### Fitting prelude

```
# Import the ARMA model
from statsmodels.tsa.arima_model import ARMA

# Instantiate the model
model = ARMA(y, order=(1,1))

# Fit the model
results = model.fit()
```

# CHAPTER 2- Fitting the Future
What lies ahead in this chapter is you predicting what lies ahead in your data. You'll learn how to use the elegant statsmodels package to fit ARMA, ARIMA and ARMAX models. Then you'll use your models to predict the uncertain future of stock prices!

### Fitting time series models
* **Creating a model:**
* The training data can be a pandas dataframe, a pandas series or a numpy array.
* Remember that the order for an ARMA model is **p**, the number of autoregressive lags, and **q**, the number of moving average lags. 

```
from statsmodels.tsa.arima_model import ARMA
model = ARMA(timeseries, order=(p,q))
```

* **To fit an AR model, we can simply use the ARMA class with q=0.**
* `ar_model = ARMA(timeseries, order=(p,0))`
* **To fit an MA model, we set p equal to zero.**
* `ma_model = ARMA(timeseries, order=(0,q))`
* To fit the model, we use the model's `.fit()` method:

```
model = ARMA(timeseries, order=(2,1))
results = model.fit()
print(results.summary())
```

#### Fit summary
* The top section includes useful information such as the order of the model that we fit, the number of observations or data points and the name of the time series.
* **S.D. of innovations:** is the standard deviation of the shock terms
* The next section of the summary shows the fitted model parameters
* first column: model coefficients
* second column: standard error in these coefficients (this is the uncertainty on the fitted coefficient values)

#### Introduction to the ARMAX models
* **Exogenous ARMA**
* One possible extension to the ARMA model is to use exogenous inputs to create the ARMAX model. 
* This means that we model the time series using other independent variables as well as the time series itself. 
* Use external variables as well as time series
* ARMAX = ARMA + linear regression
* ARMAX is like a combination between an ARMA model and a normal linear regression model 

* **Equation for simple ARMA(1,1) model:**
* $y_t = a_1y_{t-1}+m_1\epsilon_{t-1}+\epsilon_t$

* **Equation for simple ARMAX(1,1) model:**
* $y_t = x_1z_t +a_1y_{t-1}+m_1\epsilon_{t-1}+\epsilon_t$

* The only difference is one extra term: $x_1z_t$
* New independent variable: $z_t$
* Its coefficient: $x_1$


* When might ARMAX be useful?
* Imagine you wanted to model your own daily personal productivity
* This could reasonably be an ARMA model as your productivity on previous days may have an effect on your productivity today (you could be overworked or just on a roll).
* A useful exogenous variable could be the amount of sleep you got the night before, since this might affect your productivity
* Here, $z_1$ would be hours slept, and if more sleep made you more productive, then the coefficient $x_1$ would be positive. 
* We can fit an ARMAX model using the same ARMA model class we used before. The only difference is that we will now feed in our exogenous variabe using the `exog` keyword.
* The model order and the fitting procedure are just the same. 

#### Fitting ARMAX

```
# Instantiate the model
model = ARMA(df['productivity'], order=(2,1), exog=df['hours_sleep'])
```

* The ARMAX summary now includes information in the $x_1$ row for the exogenous variable parameter.

#### Forecasting
* Now that we know how to fit models, let's use them to predict the future. 
* $y_t = a_1y_{t-1} + \epsilon_t$
* $y_t = 0.6 * 10 + \epsilon_t$
* $y_t = 6.0 + \epsilon_t$

* If the shock term had a standard deviation of 1, we would predict our lower and upper uncertainty levels to be 5 and
7

* **$5.0<y_t<7.0$**
* In the time period we have data for, we can make lots of these predictions in-sample; using the previous series value to extimate the next ones. This is called a **one-step_ahead prediction.** This allows us to evaluate how good out model is at predicting just one value ahead. 
* The uncertainty is due to the random shock terms that we can't predict

* Previously we had been using the ARMA model class from statsmodels.

## SARIMAX
* The SARIMAX model class can do everything that the ARMA model class can do and can be extended even further
* This class has an additional model order for **Seasonality**
* For now, a SARIMAX model with order **p, 0, q** is exactly the same as an **ARMA(p, q)** model. 
* We can also add a constant to the model using trend equals c.
* If the time series isn't centered around zero, the constant c is a must. 

```
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Just an ARMA(p,q) model
model = SARIMAX(df, order=(p, 0, q))
```

#### Making one-step-ahead predictions
* Starting from a SARIMAX fitted results object, we can use its `get_prediction` method to generate in-sample predictions. 

```
# Make predictions for last 25 values
results = model.fit()
# Make in-sample prediction
forecast = results.get_prediction(start=-25)
# Get confidence intervals of forecasts
confidence_intervals = forecast.conf_int()
```
* We set the start parameter as a negatice integer stating how many steps back to begin the forecast.
* Setting start to -25 means we make predictions for the last 25 entries of the training data. 
* This returns a forecast object
* The central value of the forecast is stored in the `predicted_mean` attribute of the forecast object
* This predicted mean is a pandas series
* To get the lower and upper limits on the values of our predictions, we use the `conf_int` method of the forecast object
* This generates a pandas DataFrame of the lower and upper certainty range of our prediction
* We can use pyplot's `.plot` method to plot the mean value:


```
plt.figure()

# Plot prediction
plt.plot(dates,
         mean_forecast.values,
         color='red',
         label='forecast')
# Shade uncertainty area
plt.fill_between(dates, lower_limits, upper_limits, color='pink')

plt.show()
```
* We use `pyplot.fill_between` to shade the area between our lower and upper limits
* We can also make predictions further than just one step ahead 

#### Dynamic predictions
* To make these dynamic predictions we predict one step ahead and use this predicted value to forecast the next value after that... and so on. 
* Since we don't know the shock term at each step, our uncertainty can grow very quickly.
* Making dynamic predictions is very similar to making one-step-ahead predictions

```
results = model.fit()
forecast = results.get_prediction(start=-25, dynamic=True)
```
* The only difference is that in the `get_predictions` method, we set the parameter `dynamic = True`

```
results = model.fit()
forecast = results.get_prediction(start=-25, dynamic=True)
#forecast mean
mean_forecast = forecast.predicted_mean
# Get confidence intervals of forecasts
confidence_intervals = forecast.conf_int()
```

* Finally, after testing our predictions in-sample, we can use our model to predict the future 
* To make future forecasts, we use the `get_forecast` method of the results object:

```
forecast = results.get_forecast(steps=20)
```

* We choose the number of steps after the end of the training data to forecast up to.
* Everything else neatly follows as before:

```
#forecast mean
mean_forecast = forecast.predicted_mean

# Get confidence intervals of forecasts
confidence_intervals = forecast.conf_int()
```
* This is also a dynamic forecast

```
# Generate predictions
one_step_forecast = results.get_prediction(start=-30)

# Extract prediction mean
mean_forecast = one_step_forecast.predicted_mean

# Get confidence intervals of predictions
confidence_intervals = one_step_forecast.conf_int()

# Select lower and upper confidence limits
lower_limits = confidence_intervals.loc[:,'lower close']
upper_limits = confidence_intervals.loc[:,'upper close']

# Print best estimate predictions
print(mean_forecast)
```

```
# plot the amazon data
plt.plot(amazon.index, amazon, label='observed')

# plot your mean predictions
plt.plot(mean_forecast.index, mean_forecast, color='r', label='forecast')

# shade the area between your confidence limits
plt.fill_between(lower_limits.index, lower_limits,
		 upper_limits, color='pink')

# set labels, legends and show plot
plt.xlabel('Date')
plt.ylabel('Amazon Stock Price - Close USD')
plt.legend()
plt.show()
```

```
# Generate predictions
dynamic_forecast = results.get_prediction(start=-30, dynamic=True)

# Extract prediction mean
mean_forecast = dynamic_forecast.predicted_mean

# Get confidence intervals of predictions
confidence_intervals = dynamic_forecast.conf_int()

# Select lower and upper confidence limits
lower_limits = confidence_intervals.loc[:,'lower close']
upper_limits = confidence_intervals.loc[:,'upper close']

# Print best estimate predictions
print(mean_forecast)
```

```
# plot the amazon data
plt.plot(amazon.index, amazon, label='observed')

# plot your mean forecast
plt.plot(mean_forecast.index, mean_forecast, color='r', label='forecast')

# shade the area between your confidence limits
plt.fill_between(lower_limits.index, lower_limits, 
         upper_limits, color='pink')

# set labels, legends and show plot
plt.xlabel('Date')
plt.ylabel('Amazon Stock Price - Close USD')
plt.legend()
plt.show()
```

#### Introduction to ARIMA models
* Tying together forecasting and non-stationarity and show how to overcome some problems that arise when these things meet.
* Remember: You cannot apply an ARMA model to non-stationary time series.
    * Take the difference of the time series
    * Once the data is stationary, we can model it with an ARMA model.
    * **However, when we do this, we will have a model which is trained to predict *the value of the difference of the time series.***
    * Obviously, what we really want to predict is not the difference, but the actual value of the time series
    * We can achieve this by careful transforming our prediction of the differences.
    
    
## Reconstructing original time series after differencing

```
diff_forecast = results.get_forecast(steps=10).predicted_mean
from numpy import cumsum
mean_forecast = cumsum(diff_forecast) + df.iloc[-1,0]
```

* We start with predictions of the difference values 
* **The opposite of taking the difference is taking the cumulative sum or integral.**
* We will need to use this transform to go from predictions of the difference values to predictions of the absolute values: `np.cumsum`
* If we apply this function, we now have a prediction of how much the time series changed from its initial value over the forecast period. 
* To get an **absolute value, we need to add the last value of the original time series to this.**
* We now have a forecast of the non-stationary time series
* If we would like to plot our uncertainties as before, we will need to carefully transform them as well.


#### ARIMA model:
* Take the difference
* Fit the ARMA model
* **Integrate the forecast**


* These steps are very common in time series modeling, and are referred to as ARIMA
* This is a lot of work, but thankfully there is an extension of the ARMA model which does this work for us.
* **This is the ARIMA model, or, Autoregressive Integrated Moving Average model.**
* We can implement an ARIMA model using the SARIMAX model class from statsmodels. 
* The ARIMA model has three model orders:
    * **P**: number of autoregressive lags (AR order)
    * **D**: order of differencing
    * **Q**: moving average order

```
from statsmodels.tsa.statespace.sarimax import SARIMAX
model = SARIMAX(df, order = (p,d,q))
```

* ARIMA(p, 0, q) = ARMA(p, q)

#### Using the ARIMA model
* Pass it in a non-differenced time series and the model order:
* Below, we want to differene the time series data just once and then apply an ARMA(2, 1) model.
* This achieved by using an ARIMA(2,1,1) model.
* Once we have stated the difference parameter, we don't need to worry about differencing any more.

```
# Create model
model = SARIMAX(df, order=(2,1,1))
# Fit model
model.fit()
# Make forecast
mean_forecast = results.get_forecast(steps=10).predicted_mean
```
* The differencing and integration steps are all taken care of by the model object
* We must still be careful about selecting the right amount of differencing
* Remember: we difference our data only until it is stationary and no more
* We will work this out before we apply our model, using the Augmented Dickey-Fuller test to decide the difference order
* By the time we come to apply a model, we already know the degree of differencing we should apply

```
# Take the first difference of the data
amazon_diff = amazon.diff().dropna()

# Create ARMA(2,2) model
arma = SARIMAX(amazon_diff, order = (2, 0, 2))

# Fit model
arma_results = arma.fit()

# Print fit summary
print(arma_results.summary())
```

```
# Make arma forecast of next 10 differences
arma_diff_forecast = arma_results.get_forecast(steps=10).predicted_mean

# Integrate the difference forecast
arma_int_forecast = np.cumsum(arma_diff_forecast)

# Make absolute value forecast
arma_value_forecast = arma_int_forecast + amazon.iloc[-1,0]

# Print forecast
print(arma_value_forecast)
```

```
# Create ARIMA(2,1,2) model
arima = SARIMAX(amazon, order=(2,1,2))

# Fit ARIMA model
arima_results = arima.fit()

# Make ARIMA forecast of next 10 values
arima_value_forecast = arima_results.get_forecast(steps=10).predicted_mean

# Print forecast
print(arima_value_forecast)
```

## CHAPTER 3- The Best of the Best Models

 You'll learn how to identify promising model orders from the data itself, then, once the most promising models have been trained, you'll learn how to choose the best model from this fitted selection. You'll also learn a great framework for structuring your time series projects.

#### Intro to ACF and PACF
* How do we choose which ARIMA model to fit?
* The model order is very important to the quality of the forecasts.
* **One of the main ways of identifying the correct model order is by using the autocorrelation function, the ACF, and the partial autocorrelation function, the PACF.**
    * **ACF**: Autocorrelation function
    * **PACF**: Partial autocorrelation function 


#### What is the ACF?
    * The autocorrelation function at lag-1 is the correlation between a time series and the same time series offset by one step.
        * lag-1 autocorrelation $\Rightarrow$ $corr(y_t,y_{t-1})$
    * The autocorrelation at lag-2 is the correlation between a time series and itself offset by two steps
        * lag-2 autocorrelation $\Rightarrow$ $corr(y_t,y_{t-2})$
    * ...
    * The autocorrelation function at lag-n is the correlation between a time series and the same time series offset by n steps
        * lag-n autocorrelation $\Rightarrow$ $corr(y_t,y_{t-n})$
* When we talk about the autocorrelation function, we mean the set of correlation values for different lags. 

#### What is the PACF?
* The partial autocorrelation is the correlation between a time series and the lagged version of itself **after we subtract the effect of correlation at smaller lags.**
    * So, **it is the correlation associated with just that particular lag.**
    
#### Using ACF and PACF to choose model order
* **By comparing the ACF and PACF for a time series we can deduce the model order.**
    * $\star$ If the amplitude of the ACF tails off with increasing lag, and the PACF cuts off after some lag p, then we have an AR(p) model.
    * $\star$ If the amplitude of the ACF cuts off after some lag q, and the amplitude of the PACF tails off then we have MA(q) model.
    * $\star$ If both the ACF and PACF tail off then we have an ARMA model.
        * In this case, we can't deduce the model orders of p and q from the plot
    
    
    
#### AR(p)
* ACF: Tails off
* PACF: Cuts off after lag p

#### MA(q)
* ACF: Cuts off after lag q
* PACF: Tails off

#### ARMA(p, q)
* ACF: Tails off
* PACF: Tails off
* In this case, we can't deduce the model orders of p and q from the plots

#### Implementation in Python

```
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# Create figure
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8,8))
# Make ACF plot
plot_acf(df, lags=10, zero=False, ax=ax1)
# Make PACF plot
plot_pacf(df, lags=10, zero=False, ax=ax2)

plt.show())
```

* To use them, we start by creating a figure with two sublots.
* Into each function we pass the time series DataFrame and the maximum number of lags we would like to see.
* We also tell it whether to show the autocorrelation at lag-0 (`zero=False`).
    * The ACF and PACF at lag-0 will always have a value of one, so we'll set this arguent to false to simplify the plot
* **NOTE!:** The time series MUST be made stationary BEFORE making these plots. 
    * $\Rightarrow$ If the ACF values are high and tail off very, very slowly, this is a sign that the data is non-stationary, and so needs to be differenced.
    * $\Rightarrow$ If the autocorrelation at lag-1 is very negative, this is a sign that we have taken the difference too many times.

```
# Import
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Create figure
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(12,8))
 
# Plot the ACF of df
plot_acf(df, lags=10, zero=False, ax=ax1)

# Plot the PACF of df
plot_pacf(df, lags=10, zero=False, ax=ax2)

plt.show()
```

```
# Create figure
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(12,8))

# Plot ACF and PACF
plot_acf(earthquake, lags=10, zero=False, ax=ax1)
plot_pacf(earthquake, lags=10, zero=False, ax=ax2)

# Show plot
plt.show()

# Instantiate model
model = SARIMAX(earthquake, order=(1,0,0))

# Train model
results = model.fit()
```

### Intro to AIC and BIC
* In the last chapter, we mentioned how ACF and PACF can't be used to choose the order of a model, when both of the orders p and q are non-zero.
* However, there are more tools we can use: the **AIC** and the **BIC**.

#### The Akaike Information Criterion (AIC)
* The AIC is a metric which tells us how good a model is. 
* A model which makes better predications is given a lower AIC score
* The AIC also penalies models which have lots of parameters
    * This means, if we set the order too high compared to the data, we will get a high AIC value
    * This stops us overfitting to the training data. 
    
#### The Bayesian Information Criterion (BIC)
* Very similar to AIC
* Models which fit the data better have lower BIC's 
* The BIC penalizes overly complex models

#### AIC vs BIC
* For both of these metrics a lower value suggests a better model. 
* The difference between these two metrics is how much they penalize model complexity
* The BIC penalizes additional model orders more than AIC, and so the BIC will sometimes suggest a simpler model
* The AIC and BIC will often choose the same model, but when they don't, we will have to make a choice. 
    * **AIC is better at choosing predictive models.**
    * **BIC is better at choose a good explanatory model.**
* After fitting a model in Python, we can find both the AIC and BIC by using the summary of the fitted model results object:

```
# Create a model
model = SARIMAX(df, order =(1,0,1))
# Fit model 
results = model.fit()
# Print fit summary 
print(results.summary())

# Print AIC and BIC
print('AIC:', results.aic)
print('BIC:', results.bic)
```
* **You can also access the AIC and BIC directly by using the `.aic` and `.bic`attributes of the fitted model results object.**
* **Being able to access the AIC and BIC directly means we can write for loops to fit multiple ARIMA models to a dataset, to find the best model order.**

#### Searching over AIC and BIC
* **You can also access the AIC and BIC directly by using the `.aic` and `.bic`attributes of the fitted model results object.**
* **Being able to access the AIC and BIC directly means we can write for loops to fit multiple ARIMA models to a dataset, to find the best model order.**

```
# Loop over AR order
for p in range(3):
    # Loop over MA order
    for q in range(3):
        # Fit model
        model = SARIMAX(df, order=(p,0,q))
        results = model.fit()
        # print the model order and the AIC/BIC values
        print(p, q, results.aic, results.bic)
```

* If we want to test a large number of model orders, we can append the model order and the AIC and BIC to a list, and later convert it to a DataFrame

```
# Loop over AR order
for p in range(3):
    # Loop over MA order
    for q in range(3):
        # Fit model
        model = SARIMAX(df, order=(p,0,q))
        results = model.fit()
        #Add order and scores to list
        order_aic_bic.append((p, q, results.aic, results.bic))
# Make DataFrame of model order and AIC/BIC scores
order_df = pd. DataFrame(order_aic_bic, columns=['p', 'q', 'aic', 'bic'])
```
* This means we can sort by the AIC score and not have to search through the orders by eye

```
# Sort by AIC
print(order_df.sort_values('aic'))
# Sort by BIC
print(order_df.sort_values('bic'))
```
* Reminder: When AIC and BIC favor different models, determine whether you'd rather a good predictive model (AIC) or a good explanatory model (BIC), and choose accordingly.

* Sometimes when searching over model orders, you will attempt to fit an order that leads to an error

```
# Fit model
model = SARIMAX(df, order=(2,0,1))
results = model.fit()

ValueError: Non-stationary starting autoregressice parameters found with 'enforce_stationarity' set to True
```
* The above ValueError tells us that we have tried to fit a model which would result in a non-stationary set of AR coefficients
* This is just a bad model for this data, and when we loop over p and q we would like to skip this one (above example)
* **We can skip these orders in our loop by using a try and except block in Python.**

#### When certain orders don't work
* We can skip certain oders that don't work in our for loop by using a **try and except block** in Python.

```
# Loop over AR order
for p in range(3):
    # Loop over MA order
    for q in range(3):
        try:
            # Fit model
            model = SARIMAX(df, order=(p, 0, q))
            results = model.fit()
            
            # Print the model order and the AIC/BIC values
            print(p, q, results.aic, results.bic)
        except:
            # Print AIC and BIC as None when fails
            print(p, q, None, None)
```

```
# Create empty list to store search results
order_aic_bic=[]

# Loop over p values from 0-2
for p in range(3):
  # Loop over q values from 0-2
    for q in range(3):
      	# create and fit ARMA(p,q) model
        model = SARIMAX(df, order=(p, 0, q))
        results = model.fit()
        
        # Append order and results tuple
        order_aic_bic.append((p, q, results.aic, results.bic))

# Construct DataFrame from order_aic_bic
order_df = pd.DataFrame(order_aic_bic, 
                        columns=['p', 'q', 'AIC', 'BIC'])

# Print order_df in order of increasing AIC
print(order_df.sort_values('AIC'))

# Print order_df in order of increasing BIC
print(order_df.sort_values('BIC'))
```
***

```
# Loop over p values from 0-2
for p in range(3):
    # Loop over q values from 0-2
    for q in range(3):
      
        try:
            # create and fit ARMA(p,q) model
            model = SARIMAX(earthquake, order = (p, 0, q))
            results = model.fit()
            
            # Print order and results
            print(p, q, results.aic, results.bic)
            
        except:
            print(p, q, None, None)     
```

### Model Diagnostics
* Our work isn't finished oce we've built our model.
* The next step is using common model diagnostics to confirm our model is behaving well. 
* After we have picked the final model, or the final few models, we should ask how good they are.
* **To diagnose our model, we focus on the residuals to the training data.**
    * The residuals are the difference between our model's one-step ahead predictions and the real values of the time series.
    * In statsmodels, the residuals over the training period can be accessed using the `.resid` attribute of the results object.
    
```
# Fit model
model = SARIMAX(df, order=(p, d, q))
results = model.fit()
# Assign residual to variable 
residuals = results.resid
```
* We might like to know, on average, how large the residuals are and, so, how far our predictions are from the true values.
* To answer this, we can calculate the mean absolute error of the residuals
* `mae = np.mean(np.abs(residuals))`
* **For an ideal model, the residuals should be uncorrelated Gaussian white noise, centered on zero.**
* The rest of our diagnostics will help us to see if this true. 

#### Create the 4 diagnostics plots
* 1) Standardized residual
* 2) Histogram plus estimated density
* 3) Normal Q-Q
* 4) Correlogram

```
results.plot_diagnostics()
plt.show()
```

#### Standardized residual
* Shows the one-step-ahead standardized residuals
* If our model is working correctly, there should be no obvious structure in the residuals

#### Histogram plus estimated density
* Shows us the distribution of the residuals 
* The histogram shows us the measured distribution
* The orange line shows us a smoothed version of this histogram (KDE; Kernel Density Estimate)
* The green line shows a normal distribution: N(0,1)
* If our model is good, the orange and green lines should be almost the same.

#### Normal Q-Q
* The normal Q-Q plot is another way to show how the distribution of the model residuals compares to a normal distribution.
* If our residuals are normally distributed, then all the points should lie along the red line, except perhaps some values at either end. 

#### Correlogram 
* The last plot is the correlogram, which is just **an ACF plot of the residuals rather than the data.**
* 95% of the correlations for lag greater than zero should not be significant.
* If there is significant correlation in the residuals, it means that there is information in the data that our model hasn't captured. 

#### Summary statistics
* Some of these plot also have accompanying test statistics in `results.summary()` tables.
* **`Prob(Q)`** is the p-value associated with the null hypothesis that the residuals have no correlation structure. 
* **`Prob(JB)`** is the p-value associated with the null hypothesis that the residuals are Gaussian normally distributed.
* If either p-value above is less than 0.05, we reject that respective null hypothesis.

```
# Fit model
model = SARIMAX(earthquake, order=(1,0,1))
results = model.fit()

# Calculate the mean absolute error from residuals
mae = np.mean(np.abs(results.resid))

# Print mean absolute error
print(mae)

# Make plot of time series for comparison
earthquake.plot()
plt.show()
```

```
# Create and fit model
model1 = SARIMAX(df, order=(3, 0, 1))
results1 = model1.fit()

# Print summary
print(results1.summary())
```

```
# Create and fit model
model = SARIMAX(df, order=(1, 1, 1))
results=model.fit()

# Create the 4 diagostics plots
results.plot_diagnostics()
plt.show()
```

### Box-Jenkins method
* In the previous lessons you've learned lots of tools and methods for working with and modeling time series
* In this lesson: learn about the best practices framework for using these tools.

#### The Box-Jenkins method
* The Box-Jenkins method is a kind of checklist for you to go from raw data to a model ready for production.
* The three main steps that stand between you and a production-ready model are:
    * identification
    * estimation 
    * model diagnostics

**Identification** $\Rightarrow$ **Estimation** $\Rightarrow$ **Model diagnostics**

#### Identification
* In the **identification** step, we explore and characterize the data to find some form of it which is appropriate to ARIMA modeling. 
* Is the time series stationary?
* What differencing will make it stationary?
* What transformations will make it stationary? 
* What values of p and q are most promising?
* **Identification tools:**
    * Plot the time series:
        * `df.plot()`
    * Use Augmented Dickey-Fuller test
        * `adfuller()`
    * Use transforms and/or differencing
        * `df.diff()`, `np.log()`, `np.sqrt()`
    * Plot ACF/PACF:
        * `plot_acf()`, `plot_pacf()`
        
#### Estimation
* Use the data to train the model coefficients
    * Done for us using `model.fit()`
* Choose between models using AIC and BIC
    * `results.aic`, `results.bic`
    
#### Model diagnostics
* Evaluate the quality of the best fitting model
* Here is where we use our test statistics and diagnostic plots to make sure the residuals are well behaved.
* Are the results uncorrelated?
* Are the residuals normally distributed?
    * `results.plot_diagnostics()`
    * `results.summary()`
    
#### Decision
* Using the information gathered from the statistical tests and plots during the diagnostic step, we need to make a decision:
    * Is the model good enough, or do we need to go back and rework it?
    
* If the residuals aren't as they should be we will go back and rethink our choices in the earlier steps
* If the residuals are okay, then we an go ahead and make forecasts

* **This should be your general project workflow when developing time series models.**
* You may have to repeat the process a few times in order to build a model that fits well.

#### Identification I

```
# Plot time series
savings.plot()
plt.show()

# Run Dicky-Fuller test
result = adfuller(savings['savings'])

# Print test statistic
print(result[0])

# Print p-value
print(result[1])
```

#### Identification II

```
# Create figure
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(12,8))
 
# Plot the ACF of savings on ax1
plot_acf(savings, lags=10, zero=False, ax=ax1)

# Plot the PACF of savings on ax2
plot_pacf(savings, lags=10, zero=False, ax=ax2)

plt.show()
```

#### Estimation

```
# Loop over p values from 0-3
for p in range(4):
  
  # Loop over q values from 0-3
    for q in range(4):
      try:
        # Create and fit ARMA(p,q) model
        model = SARIMAX(savings, order=(p,0,q), trend='c')
        results = model.fit()
        
        # Print p, q, AIC, BIC
        print(p, q, results.aic, results.bic)
        
      except:
        print(p, q, None, None)
```



## Chapter 4: Seasonal ARIMA Models
Learn how to use seasonal ARIMA models to fit more complex data. Learn how to decompose this data into seasonal and non-seasonal parts and then get a change to utilize all your ARIMA tools on one last global forecast challenge.

#### Seasonal Time Series
* ARIMA models can be used to predict a great many types of time series
* In this chapter, learn about **seasonal time series.**
* Seasonal data has predictable and repeated patterns.
* Repeats after any amount of time.
    * These seasonal cycles might repeat every year, every week, every hour, etc. ...
    
* **We can think of any time series as being made of 3 parts:**
    * The trend
    * The seasonal component
    * The residual
* **The full time series is these three parts added together.**

#### Seasonal decomposition using statsmodels

```
from statsmodels.tsa.seasonal import seasonal_decompose
# Decompose data
decomp_results = seasonal_decompose(df['IPG3113N'], period=12)
```
* We use the function by passing in the series
* We also need to set the period parameter, which is the number of data points in each repeated cycle. In the example above, our cycle repeats every 12 steps.
* This function retuns a decompose-results object:
* `type(decomp_results)`
* `statsmodels.tsa.seasonal.DecomposeResult`
* We can use the plot method of this object to plot the components

```
# Plot decomposed data
decomp_results.plot()
plt.show()
```
* In order to decompose the data, we need to know how often the cycles repeat.
* Often, you'll be able to guess this, but we can also use the ACF to identify the period.
* The ACF shows a **periodic correlation pattern.**
* To find the **period**, we look for a lag greater than one, which is a peak in the ACF plot.
* For example, if the first peak after lag 1 is at 12 lags, the seasonal component repeats every 12 time steps

* Sometimes it can be hard to tell by eye whether a time series is seasonal or not
    * This is where the ACF is particularly useful. 
    
#### Detrending time series

* We have detrended time series before by taking the difference.
* However, this time we are only trying to find the period of the time series, and the ACF plot will be clearer if we just subtract the rolling mean. 
* **Rememver that you can detrend by subtracting the moving average using a window size of any value bigger than the likely period.**

```
# Subtract long rolling average over N steps
df = df - df.rolling(N).mean()
# Drop NaN values
df = df.dropna()
```

* The time series is now stationary and it will be easier to interpret the ACF plot 
* We plot the ACF of the detrended data and we can clearly see that there is a seasonal period of 12 steps

```
# Create figure
fig, ax = plt.subplots(1, 1, figsize=(8,4))

# Plot ACF
plot_acf(df.dropna(), ax=ax, lags=25, zero=False)
plt.show()
```

* Since the data is seasonal, we will always have correlated residuals left if we try to fit an ARIMA model to it (meaning we aren't using all of the information in the data, and so we aren't making the best predictions possible). 

```
# Import seasonal decompose
from statsmodels.tsa.seasonal import seasonal_decompose

# Perform additive decomposition
decomp = seasonal_decompose(milk_production['pounds_per_cow'], 
                            period=12)

# Plot decomposition
decomp.plot()
plt.show()
```
```
# Create figure and subplot
fig, ax1 = plt.subplots()

# Plot the ACF on ax1
plot_acf(water['water_consumers'], lags=25, zero=False,  ax=ax1)

# Show figure
plt.show()
```
```
# Subtract the rolling mean
water_2 = water - water.rolling(15).mean()

# Drop the NaN values
water_2 = water_2.dropna()

# Create figure and subplots
fig, ax1 = plt.subplots()

# Plot the ACF
plot_acf(water_2['water_consumers'], lags=25, zero=False, ax=ax1)

# Show figure
plt.show()
```

## SARIMA models
* Seasonal ARIMA = SARIMA
* The tool of choice for a seasonal time series
* Previously we saw that we could split up our time series into a seasonal component and some non-seasonal components.
* Fitting a SARIMA model is like fitting two different ARIMA models at once: one to the seasonal part and another to the non-seasonal part
* **Since we have these two models, we will have these two models we will have two sets of orders.**
    * **`SARIMA(p,d,q)(P,D,Q)`**
    * SARIMA$(p, d, q)(P, D, Q)_S$
    
* We have non-seasonal orders for the autoregressive, difference, and moving average parts: (p, d, q)
* **Non-seasonal orders:**
    * **p**: autoregressive order
    * **d**: differencing order
    * **q**: moving average order
* **Seasonal Orders:**
    * **P**: seasonal autoregressive order
    * **D**: seasonal differencing order
    * **Q**: seasonal moving average order
    * **S**: number of time steps per cycle
 
* **Note that there is also a new order, S, which is the length of the seasonal cycle.**

#### ARIMA vs SARIMA
* Simple ARIMA(2,0,1) model equation:
    * $y_t = a_1y_{t-1}+a_2y_{t-2}+m_1\epsilon_{t-1}+\epsilon_t$
    
    
* Simple SARIMA$(0,0,0)(2,0,1)_7$ model equation:
    * $y_t = a_7y_{t-7}+a_{14}y_{y-14}+m_7\epsilon_{t-7}+\epsilon_t$


* Above is the equation for a simple SARIMA model with season length of 7 days
* Note: This SARIMA model only has a seasonal part; we have set the non-seasonal order to zero.
* This particular SARIMA model will be able to capture seasonal weekly patterns, but won't be able to capture loval, day to day patterns.
* **If we construct a SARIMA model and include non-seasonal orders as well, then we can capture both of these patterns.**
* Fitting a SARIMA model is almost the same as fitting an ARIMA model. 

#### Fitting a SARIMA model
* Fitting a SARIMA model is almost the same as fitting an ARIMA model.
* **The only difference is that we have to specify the seasonal order, as well as the regular order, when instantiating the model.**
* This means that **there are a lot of model orders we need to find.**

```
from statsmodels.tsa.statespace.sarimax import SARIMAX
# Instantiate model 
model = SARIMAX(df, order=(p,d,q), seasonal_order=(P,D,Q,S))
# Fit model
results = model.fit()
```
* In the last lesson, we learned how to find the seasonal period, S using the ACF
* The next task is to find the order of differencing

#### Seasonal differencing
* To make a time series stationary, we may need to apply seasonal differencing.
* Subtract the time series value of one season ago. 
* In seasonal differencing, instead of subtracting the most recent time series value, we subtract the time series value from one cycle ago. 

```
# Take the seasonal difference
df_diff = df.diff(S)
```
* This time, we pass in an **integer S, the length of the seasonal cycle.**
* If the time series shows a trend, then we take the normal difference.
* If there is a strong seasonal cycle, then we will also take the seasonal difference.
* Once we have found the two orders of differencing, and made the time series stationary, we need to find the other model orders.
* To find the non-seasonal orders, we plot the ACF and the PACF of the differenced time series.
* To find the seasonal orders, we plot the ACF and PACF of the differenced time series at multiple seasonal steps.
* Then, we can use the same table of ACF and PACF rules to work out the seasonal order. 

#### Plotting seasonal ACF and PACF

```
# Create figure
fig, (ax1, ax2) = plt.subplots(2,1)
# Plot seasonal ACF
plot_acf(df_diff, lags=[12,24,36,48,60,72], ax=ax1)
# Plot seasonal PACF
plot_pacf(df_diff, lags=[12,24,36,48,60,72], ax=ax2)
plt.show()
```
* This time we set the lags parameter to a list of lags instead of a maximum
* This plots the ACF and PACF at these specific lags only 

```
# Create a SARIMAX model
model = SARIMAX(df1, order=(1,0,0), seasonal_order=(1,1,0,7))

# Fit the model
results = model.fit()

# Print the results summary
print(results.summary())
```

```
# Create a SARIMAX model
model = SARIMAX(df2, order=(2,1,1), seasonal_order=(1,0,0,4))

# Fit the model
results = model.fit()

# Print the results summary
print(results.summary())
```
```
# Take the first and seasonal differences and drop NaNs
aus_employment_diff = aus_employment.diff().diff(12).dropna()
```

```
# Create the figure 
fig, (ax1, ax2) = plt.subplots(2,1,figsize=(8,6))

# Plot the ACF on ax1
plot_acf(aus_employment_diff, lags=11, zero=False, ax=ax1)

# Plot the PACF on ax2
plot_pacf(aus_employment_diff, lags=11, zero=False, ax=ax2)

plt.show()
```

```
# Make list of lags
lags = [12, 24, 36, 48, 60]

# Create the figure 
fig, (ax1, ax2) = plt.subplots(2,1,figsize=(8,6))

# Plot the ACF on ax1
plot_acf(aus_employment_diff, zero=False, lags=lags, ax=ax1)

# Plot the PACF on ax2
plot_pacf(aus_employment_diff, zero=False, lags=lags, ax=ax2)

plt.show()
```

### Automation and Saving
* A powerful tool to search over model orders plus how to save the good models you've created so you can update them later if you want to.

#### Searching over model orders
* Previously, we searched over ARIMA model orders using for-loops
* Now that we have seasonal orders as well, this is very complex
* **Fortunately, there is a package that will do most of this work for us: he `pmdarima` package**
* The **`auto_arima()`** function from this package loops over model orders to find the best one. 
* The object returned by the function is the results object of the best model found by the search. 
    * This object is almost exactly like a statsmodels SARIMAX results object and has the `.summary()` and the `.plot_diagnostics()` method as well.

```
import pmdarima as pm
results = pm.auto_arima(df)
```

#### Non-seasonal search parameters
* The `auto_arima()` function has a lot of parameters that we may want to set.
    * Many parameters have default values.
    * The only *required* argument to the function is **data**.
    * Optionally, we can also set:
        * Non-seasonal difference order
        * Initial estimates of the non-seasonal orders
            * Initial guess for p
            * Initial guess for q
        * Maximum values of non-seasonal orders to test
                
```
import pmdarima as pm
results = pm.auto_arima(df,                                          # data
                        d = 0,                                       # non-seasonal difference order
                        start_p = 1,                                 # initial guess for p
                        start_q = 1,                                 # initial guess for q
                        max_p = 3,                                   # max_value of p to test
                        max_q = 3,                                   # max value of q to test
                        )                        
```

#### Seasonal search parameters
* Must set `seasonal` argument to `True`
* Required: data/dataframe
* Non-seasonal arguments
* Length of the seasonal period 
* Order of seasonal differencing


```
results = pm.auto_arima(df,                                          # data
                        d = 0,                                       # non-seasonal difference order   
                        start_p = 1,                                 # initial guess for p
                        start_q = 1,                                 # initial guess for q
                        max_p = 3,                                   # max value of p to test
                        max_q = 3,                                   # max value of q to test
                        seasonal = True,                             # is the time series seasonal
                        m = 7,                                       # the seasonal period
                        D = 1,                                       # seasonal difference order
                        start_P = 1,                                 # initial guess for P
                        start_Q = 1,                                 # initial guess for Q
                        max_P = 2,                                   # max value of P to test
                        max_Q = 2,                                   # max value of Q to test
                        )                        
```

* **Finally, there are a few non-order parameters that we may want to set:**

```
results = pm.auto_arima(df,                                          # data
                        d = 0,                                       # non-seasonal difference order   
                        start_p = 1,                                 # initial guess for p
                        start_q = 1,                                 # initial guess for q
                        max_p = 3,                                   # max value of p to test
                        max_q = 3,                                   # max value of q to test
                        seasonal = True,                             # is the time series seasonal
                        m = 7,                                       # the seasonal period
                        D = 1,                                       # seasonal difference order
                        start_P = 1,                                 # initial guess for P
                        start_Q = 1,                                 # initial guess for Q
                        max_P = 2,                                   # max value of P to test
                        max_Q = 2,                                   # max value of Q to test
                        information_criterion = 'aic',               # used to select best model
                        trace = True,                                # print results while training
                        error_action = 'ignore',                     # ignore orders that don't work
                        stepwise = True,                             # apply intelligent order search
                        )                        
```

#### Saving model objects
* Once you have fit a model in this way, you may want to save it and load it later
* Do this using the `joblib` package

```
import joblib
# Select a filepath
filepath = 'localpath/great_model.pkl'

# Save model to filepath
joblib.dump(model_results_object, filepath)
```
* Later on, when we want to make new predictions, we can load this model again, like so:

```
# Select a filepath
filepath = 'localpath/great_model.pkl'

# Load model object from filepath
model_results_object = joblib.load(filepath)
```

#### Updating model
* Some time may have passed since we trained the save model, and we may want to incorporate data that we have collected since then.
* We can do this using the `pmdarima` model's `.update()` method:

```
# Add new observations and update parameters
model_results_object.update(df_new)
```
* This adds the new observations in `df_new` and updates the model parameters
* **This isn't the same as choosing the model order again** and so if you are updating with a large amount of new data, it may be best to back back to the start of the Box-Jenkins method.
* **Updated time series models with new data is really important since they use the most recent available data for future predictions.**